# 07: Memory-Enhanced BS Detection

Add persistent memory to remember claims, learn patterns, and build context over time.

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.m7_memory import (
    memory_enhanced_check,
    SimpleMemoryManager,
    clear_memory,
    MEMORY_STORE
)
from modules.utils import render_mermaid_diagram
from config.llm_factory import LLMFactory
from IPython.display import display

# Initialize LLM once for all cells
llm = LLMFactory.create_llm()
print("✅ Memory-enhanced BS detector ready!")
print(f"✅ LLM initialized: {type(llm).__name__}")

✅ Memory-enhanced BS detector ready!
✅ LLM initialized: ChatOpenAI


## How Memory Works

In [2]:
memory_flow = """
graph LR
    A[New Claim] --> B[Extract Entities]
    B --> C[Search Memory]
    C --> D{Found Context?}
    D -->|Yes| E[Enhanced Prompt]
    D -->|No| F[Standard Prompt]
    E --> G[LLM Analysis]
    F --> G
    G --> H[Store Result]
    H --> I[Update Patterns]
"""

display(render_mermaid_diagram(memory_flow))

## 1. Basic Memory Usage

First, let's see how memory works with a simple example.

In [3]:
# Clear memory for fresh start
clear_memory()

# First claim - no memory context
claim1 = "The Boeing 747 has four engines"
result1 = memory_enhanced_check(claim1, llm)

print(f"Claim: {claim1}")
print(f"Verdict: {result1['verdict']} ({result1['confidence']}%)")
print(f"Reasoning: {result1['reasoning'][:100]}...")
print(f"\nEntities extracted: {result1.get('related_entities', [])}")

Claim: The Boeing 747 has four engines
Verdict: LEGITIMATE (95%)
Reasoning: The Boeing 747 is a large, wide-body commercial jet airliner that is well-known for having four engi...

Entities extracted: ['The Boeing']


In [4]:
# Second claim - should use memory about 747
claim2 = "The Boeing 747 can fly at Mach 2 like the Concorde"
result2 = memory_enhanced_check(claim2, llm)

print(f"Claim: {claim2}")
print(f"Verdict: {result2['verdict']} ({result2['confidence']}%)")
print(f"Reasoning: {result2['reasoning'][:100]}...")

if result2.get('memory_context'):
    print(f"\n📝 Memory Context Used:")
    print(result2['memory_context'])

Claim: The Boeing 747 can fly at Mach 2 like the Concorde
Verdict: BS (98%)
Reasoning: The Boeing 747 is a subsonic commercial airliner with a typical cruising speed of around Mach 0.85. ...

📝 Memory Context Used:
Related previous claims:\n- The Boeing 747 has four engines: LEGITIMATE (95%)


## 2. Entity Extraction

The memory system automatically extracts entities from claims.

In [5]:
# Test entity extraction with different claims
test_claims = [
    "The Wright Brothers flew at Kitty Hawk in 1903",
    "SpaceX launched Falcon 9 from Cape Canaveral",
    "The A380 is manufactured by Airbus"
]

print("Entity Extraction Examples:\n")
for claim in test_claims:
    entities = SimpleMemoryManager.extract_entities(claim)
    print(f"Claim: {claim}")
    print(f"Entities: {entities}")
    print()

Entity Extraction Examples:

Claim: The Wright Brothers flew at Kitty Hawk in 1903
Entities: ['The Wright Brothers', 'Kitty Hawk']

Claim: SpaceX launched Falcon 9 from Cape Canaveral
Entities: ['Cape Canaveral', 'Falcon']

Claim: The A380 is manufactured by Airbus
Entities: ['A380', 'Airbus']



## 3. Building Knowledge Over Time

Watch how the system builds knowledge about the A380.

In [6]:
# Series of related claims about the A380
aviation_claims = [
    "The Airbus A380 is the world's largest passenger airliner",
    "The A380 has two full decks",
    "Emirates operates the most A380s",
    "The A380 can carry 1000 passengers"  # This is BS - max is ~850
]

print("Processing related claims about the A380:\n")
for i, claim in enumerate(aviation_claims, 1):
    result = memory_enhanced_check(claim, llm)
    
    print(f"{i}. {claim}")
    print(f"   → {result['verdict']} ({result['confidence']}%)")
    
    if result.get('similar_claims'):
        print(f"   → Found {result['similar_claims']} related claims in memory")
    print()

Processing related claims about the A380:

1. The Airbus A380 is the world's largest passenger airliner
   → LEGITIMATE (95%)

2. The A380 has two full decks
   → LEGITIMATE (95%)
   → Found 1 related claims in memory

3. Emirates operates the most A380s
   → LEGITIMATE (95%)

4. The A380 can carry 1000 passengers
   → BS (90%)
   → Found 2 related claims in memory



## 4. Cross-Reference Check

Test how memory helps when comparing entities.

In [7]:
# Check a claim that references multiple stored entities
cross_ref_claim = "The A380 is bigger than the Boeing 747"
result = memory_enhanced_check(cross_ref_claim, llm)

print(f"Cross-reference claim: {cross_ref_claim}")
print(f"\nVerdict: {result['verdict']} ({result['confidence']}%)")
print(f"Entities found: {result.get('related_entities', [])}")

if result.get('memory_context'):
    print(f"\n📝 Memory Context:")
    print(result['memory_context'])

Cross-reference claim: The A380 is bigger than the Boeing 747

Verdict: LEGITIMATE (90%)
Entities found: ['A380', 'Boeing']

📝 Memory Context:
Related previous claims:\n- The Airbus A380 is the world's largest passenger airliner: LEGITIMATE (95%)\n- The A380 has two full decks: LEGITIMATE (95%)\n- The A380 can carry 1000 passengers: BS (90%)


## 5. Pattern Detection

The system learns to recognize BS patterns over time.

In [8]:
# Submit claims with common BS patterns
bs_pattern_claims = [
    "A new quantum engine can achieve light speed",
    "Scientists created an anti-gravity drive", 
    "A perpetual motion engine was patented",
    "Another quantum breakthrough defies physics"  # Should detect pattern
]

print("Testing pattern detection:\n")
for claim in bs_pattern_claims:
    result = memory_enhanced_check(claim, llm)
    print(f"Claim: {claim}")
    print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
    
    # Check if pattern was detected
    if result.get('memory_context') and 'pattern' in result['memory_context'].lower():
        print("⚠️  BS pattern detected!")
    print()

Testing pattern detection:

Claim: A new quantum engine can achieve light speed
Verdict: BS (95%)

Claim: Scientists created an anti-gravity drive
Verdict: BS (95%)

Claim: A perpetual motion engine was patented
Verdict: LEGITIMATE (90%)

Claim: Another quantum breakthrough defies physics
Verdict: BS (85%)



## 6. Memory Statistics

View what the system has learned.

In [9]:
# View memory statistics
print("🧠 Memory Statistics:")
print(f"\nTotal claims processed: {len(MEMORY_STORE['claims'])}")

# Top entities by frequency
print(f"\nTop entities (by frequency):")
entity_counts = [(entity, len(indices)) for entity, indices in MEMORY_STORE['entities'].items()]
entity_counts.sort(key=lambda x: x[1], reverse=True)
for entity, count in entity_counts[:10]:
    print(f"  - {entity}: mentioned {count} times")

# BS patterns detected
if MEMORY_STORE['patterns']:
    print(f"\nBS patterns detected:")
    for pattern, count in MEMORY_STORE['patterns'].items():
        print(f"  - '{pattern}': found {count} times")

# Recent claims
print(f"\nRecent claims:")
for claim_record in MEMORY_STORE['claims'][-5:]:
    claim_text = claim_record['claim'][:50] + '...' if len(claim_record['claim']) > 50 else claim_record['claim']
    print(f"  - {claim_text} → {claim_record['verdict']}")

🧠 Memory Statistics:

Total claims processed: 11

Top entities (by frequency):
  - A380: mentioned 4 times
  - The Boeing: mentioned 2 times
  - Concorde: mentioned 1 times
  - Mach: mentioned 1 times
  - The Airbus: mentioned 1 times
  - Emirates: mentioned 1 times
  - Boeing: mentioned 1 times
  - Scientists: mentioned 1 times
  - Another: mentioned 1 times

BS patterns detected:
  - 'quantum': found 2 times
  - 'light speed': found 1 times
  - 'anti-gravity': found 1 times

Recent claims:
  - The A380 is bigger than the Boeing 747 → LEGITIMATE
  - A new quantum engine can achieve light speed → BS
  - Scientists created an anti-gravity drive → BS
  - A perpetual motion engine was patented → LEGITIMATE
  - Another quantum breakthrough defies physics → BS


## Interactive Testing

Try your own claims to see how memory affects the results.

In [10]:
# Test your own claim
your_claim = "The Concorde could cross the Atlantic in under 3 hours"

result = memory_enhanced_check(your_claim, llm)

print(f"Your claim: {your_claim}")
print(f"\nVerdict: {result['verdict']} ({result['confidence']}%)")
print(f"Reasoning: {result['reasoning']}")

if result.get('memory_context'):
    print(f"\n📝 Memory Context:")
    print(result['memory_context'])

print(f"\nEntities extracted: {result.get('related_entities', [])}")

Your claim: The Concorde could cross the Atlantic in under 3 hours

Verdict: LEGITIMATE (95%)
Reasoning: The Concorde was a supersonic passenger airliner capable of cruising at speeds over Mach 2, approximately 1,354 mph (2,180 km/h). Given the typical distance across the Atlantic Ocean between London and New York is about 3,460 miles (5,570 km), the Concorde was able to complete the journey in under 3 hours, often around 2 hours and 50 minutes. This speed was significantly faster than conventional subsonic aircraft, which typically take about 7-8 hours for the same route.

Entities extracted: ['Atlantic', 'The Concorde']


## Key Takeaways

1. **Entity Extraction** - Automatically identifies key entities (Boeing, A380, etc.)
2. **Context Building** - Each claim enriches the knowledge base
3. **Pattern Learning** - Detects recurring BS patterns (quantum, perpetual motion)
4. **Smart Retrieval** - Finds relevant prior claims for better verdicts
5. **Progressive Learning** - The system gets smarter with each use

### Memory Benefits:
- **Consistency** - Similar claims get similar treatment
- **Context Awareness** - Previous knowledge informs new verdicts
- **Pattern Recognition** - Identifies common BS indicators
- **Efficiency** - Leverages past analysis for faster decisions

The memory system transforms a stateless BS detector into an intelligent system that learns and improves!